In [1]:
import pandas as pd
import os
import re

In [2]:
def def_schwerpunkte(index):
    dict_schwerpunkte = {}
    for thema in dict_themen:
        dict_schwerpunkte[thema] = tabelle_kandidierende[thema][index]  # existing key, overwrite
    highest = max(dict_schwerpunkte.values())
    if highest > 1:
        try:
            schwerpunkt = ([key for key in dict_schwerpunkte if dict_schwerpunkte[key] == highest][2])+", "+([key for key in dict_schwerpunkte if dict_schwerpunkte[key] == highest][1])+", "+([key for key in dict_schwerpunkte if dict_schwerpunkte[key] == highest][0])
        except:
            try:
                schwerpunkt = ([key for key in dict_schwerpunkte if dict_schwerpunkte[key] == highest][1])+", "+([key for key in dict_schwerpunkte if dict_schwerpunkte[key] == highest][0])
            except:
                try:
                    schwerpunkt = ([key for key in dict_schwerpunkte if dict_schwerpunkte[key] == highest][0])
                except:
                    schwerpunkt = ""
    else:
        schwerpunkt = ""
    return(schwerpunkt)

In [43]:
#Hier werden Themen und Wortlisten definiert, die für ein Thema typisch sind
dict_themen = {
"Patriotismus":[
        "Heimat*",
        "Nation ",
        "eidgenössisch",
        "Freiheit",
        "Tradition",
        "Souveränität",
        "Zusammenhalt",
        "Sicherheit",
        "Demokratie",
        "Patriotismus",
        "freie.*Schweiz",
        "sichere.*Schweiz",
        "unabhängige.*Schweiz",
        "Sonderfall",
        "Neutralität",
        "Schweizer",
    ],
"Zuwanderung":
    [
        "Migration",
        "[k|K]riminelle[|r] Ausländer",
        "Ausschaffung",
        "Masseneinwanderung",
        "Bevölkerungsaustausch",
        "Härtefallklausel",
        "Gefährder",
        "Asyl",
        "Flüchtling",
        "Europa",
        "EU",
        "Beziehung",
        "Zusammenarbeit",
        "Fachkräftemangel",
        "Bevölkerungswachstum",
        "Zuwanderung",
    ],

"Umwelt":
    [
        "Umweltschutz",
        "Nachhaltigkeit",
        "Klimawandel",
        "Biodiversität",
        "Ökologie",
        "ökologisch",
        "Naturschutz",
        "Ressourcen",
        "Kreislaufwirtschaft",
        "Verkehrswende",
        "Fussabdruck",
        "grün",
        "erneuerbar",
        "umweltfreundlich",
        "Klimakrise",
        "Treibhausgas*",
        "CO2",
        "ÖV",
        "Antomausstieg", #sic!
        "Klimaschutz",
        "Klimastreik",
    ],
"Energie":
    [
        "Kernkraft*",
        "Atom*",
        "[e|E]rneuerbare[|n] Energien",
        "Energie*",
        "Solarenergie",
        "Energiewende",
        "erneuerbar",
        "Klimakrise",
        "Energieversorgung",
        "Gas*",
        "Wasserkraft*",
        "AKW",
        ".*Kraftwerk",
        "Antomausstieg" #sic!
    ],
"Wirtschaft":
    [
        "Wirtschaftswachstum",
        "Arbeitsmarkt",
        "Export",
        "Arbeitsplätze",
        "Investition",
        "Innovation",
        "Wettbewerb",
        "Handel",
        "Steuer",
        "Regulierung",
        "Industrie",
        "starke Wirtschaft",
        "KMU",
        "Fachkräftemangel",
        "Wirtschaftsstandort",
        "Ausbildung",
        "Wohlstand",
        "Unternehmen",
        "Fachkräftemangel"
    ],
"Sozialwerke":
    [
        "AHV",
        "IV",
        "Rente",
        "[b|B]enachteiligt",
        "Altersvorsorge",
        "Prämien",
        "Gesundheitskosten",
        "Krankenkasse",
        "Sozialwerk",
    ],
"Familie":
    [
    "Eltern*",
    "Kinderbetreuung",
    "Erziehung",
    "Gleichstellung",
    "Vereinbarkeit",
    "Kinderzulagen",
    "Familien*",
    "Kinder*",
    "Kita",
    "Generationen",
    "Mutter",
    "Vater",
    "Elternzeit",
    ],
"Gesundheit":
    [
    "Krankenkasse",
    "Krankenversicherung",
    "Prämien*",
    "Grundversicherung",
    "Kostenbeteiligung",
    "Versicherungspflicht",
    "Gesundheitskosten",
    "Leistungskatalog",
    "Versicherungswesen",
    "Krankenkassenprämien",
    "Risikoaufschlag",
    "Kostendämpfung",
    "Solidaritätsprinzip",
    "Eigenbeteiligung",
    "Versicherungsmarkt",
    "Prämienverbilligung",
    "Versicherungsprämie",
    "Gesundheitsreform",
    "Zusatzversicherung",
    "Spital",
    "Einheitskrankenkasse",
    "Gesundheit*",
    "Psychiatrie"
    ],
"Gleichstellung":
    [
    "Gleichstellung",
    "Frauenrechte",
    "Chancengleichheit",
    "Diskriminierung",
    "Fremdenhass",
    "rassistisch",
    "Vereinbarkeit",
    "Diversität",
    "[g|G]eschlechter",
    "Schutz der Schwächeren",
    #"^Gender*(?!.*Irrsinn)",
    ],
"Landwirtschaft":
    [
    "Bauer",
    "Landwirtschaft",
    "Nahrung",
    "Versorgungssicherheit",
    ],
"Verkehr":
    [
    "Verkehr",
    "ÖV",
    "Auto",
    "Strassen",
    "Zug",
    "Infrastruktur"
    ],
"Bildung":
    [
    "Bildung",
    "Schule",
    "Lehrer",
    "Ausbildung",
    "Beruf"
    ],
}


### Für alle gesamthaft

In [17]:
#Die Liste mit den Kandidierenden und ihren Antworten importieren
tabelle_kandidierende = pd.read_csv('kandidierende.csv') 
#Leere Felder mit "" füllen, damit später der Text zusammengesetzt werden kann
tabelle_kandidierende.fillna("", inplace=True)
tabelle_kandidierende = tabelle_kandidierende[['Name','Vorname','Partei','ListeNr','svy_q2','svy_q3','svy_q4']]

#Spalte "Schwerpunkte" hinzufügen
tabelle_kandidierende = tabelle_kandidierende.assign(Patriotismus=0.0,Zuwanderung=0.0,Umwelt=0.0,Energie=0.0,Wirtschaft=0.0,Sozialwerke=0.0,Familie=0.0,Gesundheit=0.0,Gleichstellung=0.0,Landwirtschaft=0.0,Verkehr=0.0,Bildung=0.0,Schwerpunkt="")

#print(dict_parteien[partei])
for index in tabelle_kandidierende.index:
    #Text aus den drei relevanten Antworten zusammensetzen
    text = tabelle_kandidierende["svy_q2"][index]+tabelle_kandidierende["svy_q3"][index]+tabelle_kandidierende["svy_q4"][index]
    #Für jedes Thema das definiert ist
    for thema in dict_themen:
        strength = 0
        #Für jedes Wort, das in der Wort-Liste zum Thema definiert wurde
        for wort in dict_themen[thema]:
            result = re.findall(wort, text)
            strength += len(result)
        if strength != 0:
            tabelle_kandidierende.at[index,thema]=strength
            tabelle_kandidierende.at[index,"Schwerpunkt"]=def_schwerpunkte(index)
    
tabelle_kandidierende.to_csv('Themen.csv', index=False)

### Nach Parteien aufgeteilt

In [5]:
dict_parteien = {"SVP":"01","SP":"02","FDP":"03","Mitte":"04","Grüne":"05","GLP":"06","EVP":"07","EDU":"8"}
for partei in dict_parteien:
    #Die Liste mit den Kandidierenden und ihren Antworten importieren
    tabelle_kandidierende = pd.read_csv('AG_MASTER_offizielle_kandidatenliste_20230816b.csv') 
    #Leere Felder mit "" füllen, damit später der Text zusammengesetzt werden kann
    tabelle_kandidierende.fillna("", inplace=True)
    tabelle_kandidierende = tabelle_kandidierende[['Name','Vorname','Partei','ListeNr','svy_q2','svy_q3','svy_q4']]

    #Spalte "Schwerpunkte" hinzufügen
    tabelle_kandidierende = tabelle_kandidierende.assign(Patriotismus=0.0,Zuwanderung=0.0,Umwelt=0.0,Energie=0.0,Wirtschaft=0.0,Sozialwerke=0.0,Familie=0.0,Gesundheit=0.0,Gleichstellung=0.0,Landwirtschaft=0.0,Verkehr=0.0,Bildung=0.0,Schwerpunkt="")

    #print(dict_parteien[partei])
    tabelle_kandidierende = tabelle_kandidierende.loc[tabelle_kandidierende['ListeNr'].str.contains(dict_parteien[partei], case=False)]
#tabelle_kandidierende = tabelle_kandidierende.loc[tabelle_kandidierende['ListeNr'].str.contains("03", case=False)]
    for index in tabelle_kandidierende.index:
        #Text aus den drei relevanten Antworten zusammensetzen
        text = tabelle_kandidierende["svy_q2"][index]+tabelle_kandidierende["svy_q3"][index]+tabelle_kandidierende["svy_q4"][index]
        #Für jedes Thema das definiert ist
        for thema in dict_themen:
            strength = 0
            #Für jedes Wort, das in der Wort-Liste zum Thema definiert wurde
            for wort in dict_themen[thema]:
                result = re.findall(wort, text)
                strength += len(result)
                #if re.findall(wort, text):
                #strength += len(re.findall)
            if strength != 0:
                #strength = strength/len(dict_themen[thema])
                #tabelle_kandidierende[thema][index] = strength
                #print(index,thema,strength)
                tabelle_kandidierende.at[index,thema]=strength
                tabelle_kandidierende.at[index,"Schwerpunkt"]=def_schwerpunkte(index)

    tabelle_kandidierende.to_csv(partei+'.csv', index=False)

### Welche Begriffe wurden innerhalb eines Themas wie oft verwendet?

In [41]:
df = pd.DataFrame({'Wort':dict_themen["Zuwanderung"], 'Anzahl':0})

#Die Liste mit den Kandidierenden und ihren Antworten importieren
tabelle_kandidierende = pd.read_csv('kandidierende.csv') 
#Leere Felder mit "" füllen, damit später der Text zusammengesetzt werden kann
tabelle_kandidierende.fillna("", inplace=True)
tabelle_kandidierende = tabelle_kandidierende[['Name','Vorname','Partei','ListeNr','svy_q2','svy_q3','svy_q4']]

#Spalte "Schwerpunkte" hinzufügen
tabelle_kandidierende = tabelle_kandidierende.assign(Patriotismus=0.0,Zuwanderung=0.0,Zusammenarbeit=0.0,Umwelt=0.0,Energie=0.0,Wirtschaft=0.0,Sozialwerke=0.0,Familie=0.0,Gesundheit=0.0,Gleichstellung=0.0,Landwirtschaft=0.0,Verkehr=0.0,Liberalismus=0.0,Schwerpunkt="")



#Jetzt werden die Antworten jedes Kandidierenden nach den Stichworten durchsucht und dementsprechend eine Gewichtung für jedes Thema zugewiesen
for index in tabelle_kandidierende.index:
    text = tabelle_kandidierende["svy_q2"][index]+tabelle_kandidierende["svy_q3"][index]+tabelle_kandidierende["svy_q4"][index]
    for item in df.index:
        #print(df['Wort'][item])
        if re.findall(df['Wort'][item], text):
            if df['Wort'][item] == "Staat":
                print(text)
            df.at[item,"Anzahl"]+=1
            #df['Anzahl'][item] += 1
df.sort_values(by=['Anzahl'], ascending=False)

,Wort,Anzahl
9,Europa,28
0,Migration,27
13,Fachkräftemangel,26
10,EU,25
11,Beziehung,19
12,Zusammenarbeit,18
7,Asyl,14
15,Zuwanderung,13
3,Masseneinwanderung,5
14,Bevölkerungswachstum,5


### Wie sind die Schwerpunkte in einer Parteien verteilt?

In [44]:
#Die Liste mit den Kandidierenden und ihren Antworten importieren
tabelle_kandidierende = pd.read_csv('kandidierende.csv') 
#Leere Felder mit "" füllen, damit später der Text zusammengesetzt werden kann
tabelle_kandidierende.fillna("", inplace=True)
tabelle_kandidierende = tabelle_kandidierende[['Name','Vorname','Partei','ListeNr','svy_q2','svy_q3','svy_q4']]

#Spalte "Schwerpunkte" hinzufügen
tabelle_kandidierende = tabelle_kandidierende.assign(Patriotismus=0.0,Zuwanderung=0.0,Umwelt=0.0,Energie=0.0,Wirtschaft=0.0,Sozialwerke=0.0,Familie=0.0,Gesundheit=0.0,Gleichstellung=0.0,Landwirtschaft=0.0,Verkehr=0.0,Bildung=0.0,Schwerpunkt="")

#print(dict_parteien[partei])
for index in tabelle_kandidierende.index:
    #Text aus den drei relevanten Antworten zusammensetzen
    text = tabelle_kandidierende["svy_q2"][index]+tabelle_kandidierende["svy_q3"][index]+tabelle_kandidierende["svy_q4"][index]
    #Für jedes Thema das definiert ist
    for thema in dict_themen:
        strength = 0
        #Für jedes Wort, das in der Wort-Liste zum Thema definiert wurde
        for wort in dict_themen[thema]:
            result = re.findall(wort, text)
            strength += len(result)
        if strength != 0:
            tabelle_kandidierende.at[index,thema]=strength
            tabelle_kandidierende.at[index,"Schwerpunkt"]=def_schwerpunkte(index)

partei = "SP"

df_partei = pd.DataFrame({'Thema': [],'Anzahl': []})
dict_parteien = {"SVP":"01","SP":"02","FDP":"03","Mitte":"04","Grüne":"05","GLP":"06","EVP":"07","EDU":"8"}

for thema in dict_themen:
    tabelle_kandidierende_themen = tabelle_kandidierende.loc[tabelle_kandidierende['ListeNr'].str.contains(dict_parteien[partei], case=False)]
    anzahl = len(tabelle_kandidierende_themen[tabelle_kandidierende_themen['Schwerpunkt'].str.contains(thema)])
    new_row = {'Thema':thema, 'Anzahl':anzahl}
    df_partei.loc[len(df_partei)] = new_row
df_partei

,Thema,Anzahl
0,Patriotismus,3.0
1,Zuwanderung,5.0
2,Umwelt,8.0
3,Energie,14.0
4,Wirtschaft,4.0
5,Sozialwerke,4.0
6,Familie,11.0
7,Gesundheit,10.0
8,Gleichstellung,0.0
9,Landwirtschaft,1.0


### Wer hat innerhalb einer Partei einen bestimmten Schwerpunkt gesetzt?

In [46]:
tabelle_kandidierende[(tabelle_kandidierende['Schwerpunkt'].str.contains('Zuwanderung')) & (tabelle_kandidierende['ListeNr'].str.contains('01'))]

,Name,Vorname,Partei,ListeNr,svy_q2,svy_q3,svy_q4,Patriotismus,Zuwanderung,Umwelt,Energie,Wirtschaft,Sozialwerke,Familie,Gesundheit,Gleichstellung,Landwirtschaft,Verkehr,Bildung,Schwerpunkt
62,Bircher,Martina,SVP,01a,Ich setzte mich für unsere Schweiz ein damit u...,Es braucht eine neue Migrations- und Asylpolit...,"Selbstbewusster Kanton, der seine Vorzüge einb...",0.0,4.0,0.0,0.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,Zuwanderung
206,Glarner,Andreas,SVP,01a,Als die Schweiz im Jahre 1992 dem EWR beitrete...,Das Asyl- und Migrationsproblem - es kommen zu...,"Ich kämpfe für eine lebenswerte, sichere, frei...",3.0,5.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,Zuwanderung
244,Hasler,Samuel,Junge_SVP,01b,"Ich möchte etwas verändern und dabei helfen, d...",Die Schweiz schlittert aktuell von einer Krise...,"Der Kanton Aargau ist zentral, wenn es um Tran...",0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Energie, Zuwanderung"
287,Hug,Ramon,Junge_SVP,01b,Die Schweiz wird von linken Politikern zerstör...,Die Politik muss sich für die Sicherheit der S...,"Ich stehe zum Föderalismus, das heisst, möglic...",3.0,7.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Zuwanderung


In [60]:
tabelle_kandidierende[(tabelle_kandidierende['Vorname'] == 'Irène') & (tabelle_kandidierende['Name'] == 'Kälin')]

,Name,Vorname,Partei,ListeNr,svy_q2,svy_q3,svy_q4,Patriotismus,Zuwanderung,Umwelt,Energie,Wirtschaft,Sozialwerke,Familie,Gesundheit,Gleichstellung,Landwirtschaft,Verkehr,Bildung,Schwerpunkt
312,Kälin,Irène,Gruene,05a,Wir haben nur diesen einen Planeten. Und wir w...,"Neben der Klimakrise, die wir meistern müssen,...",Der Aargau ist ein Kanton der Regionen und dar...,1.0,3.0,4.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Energie


# Codeschnippsel

In [18]:
tabelle_kandidierende

,Name,Vorname,Partei,ListeNr,svy_q2,svy_q3,svy_q4,Patriotismus,Zuwanderung,Umwelt,Energie,Wirtschaft,Sozialwerke,Familie,Gesundheit,Gleichstellung,Landwirtschaft,Verkehr,Bildung,Schwerpunkt
0,Abrham,Sarah,EVP,07g,,,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,Abt,Lukas,Junge_SVP,01b,,,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,Ackermann,Adrian,FDP,03e,,,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,Acklin-Stihl,Jasmin,Junge_EVP,07b,In der Politik gibt es für mich drei Herzensan...,Neben der Bildungspolitik ist ein grosser Baus...,- Verbesserung der Gesundheitspolitik - Offenh...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0,0.0,4.0,Bildung
4,Aegerter,Emanuel,GLP,06g,,,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,Zingg,Lionel,FDP,03b,,,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
709,Zobrist,Nico Julian,SP,02f,Ich will mich auch in Bern einsetzen für eine ...,"Die Klimakrise ist unser grösstes Problem, das...",Als Nationalrat steht das Wohle der gesamten S...,0.0,1.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,Umwelt
710,Züger Cáceres,Regula,Gruene,05c,,,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
711,Zumstein,Antonia,GLP,06c,"Der Erhalt unserer wertvollen Natur, das fried...",Der Klimawandel geht an der Schweiz nicht spur...,Die Ausrichtung des Aargaus in der Energieprod...,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Energie


,Name,Vorname,Partei,ListeNr,svy_q2,svy_q3,svy_q4,Patriotismus,Zuwanderung,Umwelt,Energie,Wirtschaft,Sozialwerke,Familie,Gesundheit,Gleichstellung,Landwirtschaft,Verkehr,Bildung,Schwerpunkt


In [30]:
tabelle_kandidierende[(tabelle_kandidierende['Schwerpunkt'] == 'Zuwanderung') | (tabelle_kandidierende['Schwerpunkt'] == 'Zuwanderung')]

,Name,Vorname,Partei,ListeNr,svy_q2,svy_q3,svy_q4,Patriotismus,Zuwanderung,Umwelt,Energie,Wirtschaft,Sozialwerke,Familie,Gesundheit,Gleichstellung,Landwirtschaft,Verkehr,Bildung,Schwerpunkt
62,Bircher,Martina,SVP,01a,Ich setzte mich für unsere Schweiz ein damit u...,Es braucht eine neue Migrations- und Asylpolit...,"Selbstbewusster Kanton, der seine Vorzüge einb...",0.0,4.0,0.0,0.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,Zuwanderung
169,Flach,Beat,GLP,06a,"Ich möchte daran mitwirken, dass die Schweiz u...","Mehr Umweltschutz, Massnahmen gegen den Klimaw...",Wichtiger als dem Aargau mehr Vorteile zu vers...,1.0,4.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Zuwanderung
170,Flach,Angélique,GLP,06g,Für mich sind alle Menschen eine Bereicherung ...,Nebst Klimaveränderungen sehe ich die grösste ...,Nur für den Kanton denken ist mir zu einseitig...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Zuwanderung
206,Glarner,Andreas,SVP,01a,Als die Schweiz im Jahre 1992 dem EWR beitrete...,Das Asyl- und Migrationsproblem - es kommen zu...,"Ich kämpfe für eine lebenswerte, sichere, frei...",3.0,5.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,Zuwanderung
209,Gökçen,Sinem,SP,02a,"Ich arbeitete mich hoch, von der Realschule zu...",Das Thema der Kaufkraft und der Migration. Ste...,Neue Integrationsangebote schaffen und bestehe...,0.0,4.0,0.0,0.0,1.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,Zuwanderung
231,Haefeli,Sophie,Juso,02b,In so unglaublich vielen Bereichen in der Schw...,Eine grosse Herausforderung stellt die Klimakr...,Beschleunigung der Asylverfahren: Lange Wartez...,0.0,4.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Zuwanderung
287,Hug,Ramon,Junge_SVP,01b,Die Schweiz wird von linken Politikern zerstör...,Die Politik muss sich für die Sicherheit der S...,"Ich stehe zum Föderalismus, das heisst, möglic...",3.0,7.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Zuwanderung
351,Kundert,Anke,GLP,06e,"Ich kandidiere, weil ich mich mit dem lösungso...",Die grösste Herausforderung sehe ich in der Pr...,Unternehmen im Aargau sollten auf ihren Weg de...,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Zuwanderung
388,Lütolf,Harry,Mitte,04f,"Ich stehe für eine familienfreundliche, umwelt...",Die heissen Eisen in den kommenden Jahren sind...,Mit meinem bisherigen politischen Wirken habe ...,0.0,4.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,Zuwanderung
419,Mezzi,Daniele,Mitte,04a,Aus einem einfachen Grund: Ich möchte am polit...,Ich erachte die Schweiz als wunderbares Land. ...,"Ich möchte, dass der Kanton Aargau in Bundesbe...",0.0,2.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Zuwanderung


In [57]:
text = "Parteiübergreifende Zusammenarbeit ist entscheidend, um gute und tragfähige Lösungen zu erzielen. Sehr gerne würde ich mich für weitere vier Jahre als Mitglied des Ständerats in aktuelle Diskussionen und Themen einbringen – immer mit Blick für den Aargau und die Schweiz.Stromversorgungssicherheit; Demografische Entwicklung insbesondere in Bezug auf unsere Wirtschafts- und Sozialversicherungssysteme; Erhalt der Funktionsfähigkeit unserer Infrastrukturen (Mobilität, Telekommunikation und Digitalisierung usw.); Verteidigungsfähigkeit Der Aargau muss in Bundesbern im Verbund mit seinen Nachbarkantonen seine Interessen intensiver und mit Nachdruck verfolgen; der Aargau muss als zentraler Energie-, Waren-, Verkehrs- und Wasserkanton in der Infrastrukturplanung des Bundes angemessener berücksichtigt und abgegolten werden."
#Für jedes Thema das definiert ist
for thema in dict_themen:
    strength = 0
    #Für jedes Wort, das in der Wort-Liste zum Thema definiert wurde
    for wort in dict_themen[thema]:
        result = re.findall(wort, text)
        strength += len(result)
    if strength != 0:
        print(thema,strength)

Zuwanderung 1
Energie 1
Verkehr 3


In [169]:
    text = tabelle_kandidierende["svy_q1"][97]+tabelle_kandidierende["svy_q2"][97]+tabelle_kandidierende["svy_q3"][97]+tabelle_kandidierende["svy_q4"][97]

In [310]:
#tabelle_kandidierende.head(50)
tabelle_kandidierende[(tabelle_kandidierende['ListeNr'] == '03a')]

,ListeNr,WahlzettelNr,Name,Vorname,Wohnort,Beruf,Jahrgang,bisher,Amtsjahr,Parteibezeichnung,...,svy_q4,dw_id,iframe_full,iframe_webcomponent,photo_file_path,has_teaser,has_portrait,photo_id_teaser,photo_id_portrait,Schwerpunkte
143,03a,1,Jauslin,Matthias Samuel,Wohlen,"Eidg. dipl. Elektroinstallateur, Unternehmer",1962,1,seit 2015,FDP.Die Liberalen,...,Im Zentrum steht eine liberale Wirtschafts- un...,44Ldb,"<iframe title=""Steckbrief Matthias Samuel Jaus...","<div style=""min-height:324px""><script type=""te...",ag_fdp_jauslin_matthias_samuel.jpg,True,True,20230816_AG_teaser_ag_fdp_jauslin_matthias_samuel,20230816_AG_portrait_ag_fdp_jauslin_matthias_s...,Patriotismus
144,03a,2,Riniker,Maja,Suhr,"Betriebsökonomin FH, Vizepräsidentin Nationalrat",1978,1,seit 2019,FDP.Die Liberalen,...,"Falls ich wiedergewählt würde, kann ich 2025 N...",kWm3c,"<iframe title=""Steckbrief Maja Riniker (bisher...","<div style=""min-height:324px""><script type=""te...",ag_fdp_riniker_maja.jpg,True,True,20230816_AG_teaser_ag_fdp_riniker_maja,20230816_AG_portrait_ag_fdp_riniker_maja,Patriotismus
145,03a,3,Glarner,Jeanine,Möriken-Wildegg,"Geschäftsführerin, Grossrätin, Gemeindeammann",1984,0,,FDP.Die Liberalen,...,Unsere Wirtschaft im Kanton Aargau ist auf ein...,k3b95,"<iframe title=""Steckbrief Jeanine Glarner"" ari...","<div style=""min-height:324px""><script type=""te...",ag_fdp_glarner_jeanine.jpg,True,True,20230816_AG_teaser_ag_fdp_glarner_jeanine,20230816_AG_portrait_ag_fdp_glarner_jeanine,Umwelt Patriotismus
146,03a,4,Schoop,Adrian,Turgi,"Dr. iur., Unternehmer, Grossrat, Gemeindeammann",1985,0,,FDP.Die Liberalen,...,Der Aargau muss der Wirtschaftsmotor der Schwe...,ECsaY,"<iframe title=""Steckbrief Adrian Schoop"" aria-...","<div style=""min-height:324px""><script type=""te...",ag_fdp_schoop_adrian.jpg,True,True,20230816_AG_teaser_ag_fdp_schoop_adrian,20230816_AG_portrait_ag_fdp_schoop_adrian,Patriotismus
147,03a,5,Freiermuth,Sabina,Zofingen,"Dir.-Ass. NSH, Grossrätin, Parteipräsidentin",1964,0,,FDP.Die Liberalen,...,In der kommenden Legislatur geht es auf nation...,Bq9MN,"<iframe title=""Steckbrief Sabina Freiermuth"" a...","<div style=""min-height:324px""><script type=""te...",ag_fdp_freiermuth_sabina.jpg,True,True,20230816_AG_teaser_ag_fdp_freiermuth_sabina,20230816_AG_portrait_ag_fdp_freiermuth_sabina,Patriotismus
148,03a,6,Meier,Titus,Brugg,"Dr. phil. Historiker, Lehrer, Grossrat",1981,0,,FDP.Die Liberalen,...,,pYCbd,"<iframe title=""Steckbrief Titus Meier"" aria-la...","<div style=""min-height:324px""><script type=""te...",ag_fdp_meier_titus.jpg,True,True,20230816_AG_teaser_ag_fdp_meier_titus,20230816_AG_portrait_ag_fdp_meier_titus,
149,03a,7,Berner,Yannick,Aarau,"M.A. HSG, Unternehmer, Vorstand Swissmem, Gros...",1992,0,,FDP.Die Liberalen,...,Der Kanton Aargau soll noch mehr als Werkplatz...,UHlbZ,"<iframe title=""Steckbrief Yannick Berner"" aria...","<div style=""min-height:324px""><script type=""te...",ag_fdp_berner_yannick.jpg,True,True,20230816_AG_teaser_ag_fdp_berner_yannick,20230816_AG_portrait_ag_fdp_berner_yannick,Patriotismus
150,03a,8,Hauser,Claudia,Döttingen,"Betriebsök. KSZH, Grossrätin, Vizeparteipräs.",1968,0,,FDP.Die Liberalen,...,,rygsZ,"<iframe title=""Steckbrief Claudia Hauser"" aria...","<div style=""min-height:324px""><script type=""te...",ag_fdp_hauser_claudia.jpg,True,True,20230816_AG_teaser_ag_fdp_hauser_claudia,20230816_AG_portrait_ag_fdp_hauser_claudia,
151,03a,9,Tüscher,Bruno,Münchwilen,"Unternehmer, Grossrat, Gemeindeammann",1984,0,,FDP.Die Liberalen,...,"- Gesicherte Altersvorsorge, damit die jetzige...",Q20bB,"<iframe title=""Steckbrief Bruno Tüscher"" aria-...","<div style=""min-height:324px""><script type=""te...",ag_fdp_tuescher_bruno.jpg,True,True,20230816_AG_teaser_ag_fdp_tuescher_bruno,20230816_AG_portrait_ag_fdp_tuescher_bruno,Patriotismus
152,03a,10,Meier,Adrian,Menziken,"Betriebsökonom FH, Geschäftsinhaber, Grossrat",1990,0,,FDP.Die Liberalen,...,Der Kanton Aargau ist ein Kanton der Regionen....,tkLv

In [408]:
#tabelle_kandidierende[(tabelle_kandidierende['Schwerpunkte'] == 'Patriotismus ')]


,ListeNr,WahlzettelNr,Name,Vorname,Wohnort,Beruf,Jahrgang,bisher,Amtsjahr,Parteibezeichnung,...,svy_q4,dw_id,iframe_full,iframe_webcomponent,photo_file_path,has_teaser,has_portrait,photo_id_teaser,photo_id_portrait,Schwerpunkte
3,01a,4,Bircher,Martina,Aarburg,"Betriebsökonomin FH, Vizeammann",1984,1,seit 2019,SVP – Schweizerische Volkspartei,...,"Selbstbewusster Kanton, der seine Vorzüge einb...",TkOrQ,"<iframe title=""Steckbrief Martina Bircher (bis...","<div style=""min-height:324px""><script type=""te...",ag_svp_bircher_martina.jpg,True,True,20230816_AG_teaser_ag_svp_bircher_martina,20230816_AG_portrait_ag_svp_bircher_martina,Gesundheit Wirtschaft Patriotismus
10,01a,11,Barp,Miro,Brugg,"Psych. Pfl. HF, Leiter Sicherheitsdienst, Dozent",1971,0,,SVP – Schweizerische Volkspartei,...,"Ich will, dass der Aargau als viertgrösster Ka...",ac0Bq,"<iframe title=""Steckbrief Miro Barp"" aria-labe...","<div style=""min-height:324px""><script type=""te...",ag_svp_barp_miro.jpg,True,True,20230816_AG_teaser_ag_svp_barp_miro,20230816_AG_portrait_ag_svp_barp_miro,Patriotismus Gesundheit Wirtschaft
36,02a,5,Brügger,Martin,Brugg,"Ing. HTL, Grossrat, Einwohnerrat",1959,0,,SP – Sozialdemokratische Partei,...,"Ich will, dass die Schweiz lebenswert ist und ...",f2q5G,"<iframe title=""Steckbrief Martin Brügger"" aria...","<div style=""min-height:324px""><script type=""te...",ag_sp_bruegger_martin.jpg,True,True,20230816_AG_teaser_ag_sp_bruegger_martin,20230816_AG_portrait_ag_sp_bruegger_martin,Patriotismus Gesundheit
41,02a,10,Capanni,Luzia,Windisch,"lic. phil. I, Fachperson Integration, Grossrätin",1978,0,,SP – Sozialdemokratische Partei,...,Die Solidarität in der Gesellschaft stärken. L...,OQBN7,"<iframe title=""Steckbrief Luzia Capanni"" aria-...","<div style=""min-height:324px""><script type=""te...",ag_sp_capanni_luzia.jpg,True,True,20230816_AG_teaser_ag_sp_capanni_luzia,20230816_AG_portrait_ag_sp_capanni_luzia,Gesundheit Wirtschaft Familie
43,02a,12,Gökçen,Sinem,Untersiggenthal,"MLaw, Juristin, Gerichtsschreiberin",1986,0,,SP – Sozialdemokratische Partei,...,Neue Integrationsangebote schaffen und bestehe...,xzol6,"<iframe title=""Steckbrief Sinem Gökçen"" aria-l...","<div style=""min-height:324px""><script type=""te...",ag_sp_goekcen_sinem.jpg,True,True,20230816_AG_teaser_ag_sp_goekcen_sinem,20230816_AG_portrait_ag_sp_goekcen_sinem,Gesundheit Wirtschaft Familie
58,02b,11,Covo,Sandro,Bremgarten,"Software Engineer, Präsident SP Bezirk Bremgarten",1997,0,,JUSO – SP Jungsozialist*innen,...,"Ich kandidiere nicht für den Kanton, sondern f...",hKCpQ,"<iframe title=""Steckbrief Sandro Covo"" aria-la...","<div style=""min-height:324px""><script type=""te...",ag_juso_covo_sandro.jpg,True,True,20230816_AG_teaser_ag_juso_covo_sandro,20230816_AG_portrait_ag_juso_covo_sandro,Gesundheit Patriotismus Umwelt
94,02d,16,Widmer,Denise,Unterentfelden,"MAS Psychologie, Gesamtleitung Kinderheim",1964,0,,SP F&J – Familie und Jugend,...,"Der Aargau ist sehr schön, vielfältig und hat ...",jL64s,"<iframe title=""Steckbrief Denise Widmer"" aria-...","<div style=""min-height:324px""><script type=""te...",ag_sp_widmer_denise.jpg,True,True,20230816_AG_teaser_ag_sp_widmer_denise,20230816_AG_portrait_ag_sp_widmer_denise,Wirtschaft Gesundheit Patriotismus
97,02e,3,Muji,Idvan,Ennetbaden,Gesundheitscoach,1989,0,,SP GfV – Gemeinsam für Vielfalt,...,,kfGur,"<iframe title=""Steckbrief Idvan Muji"" aria-lab...","<div style=""min-height:324px""><script type=""te...",ag_sp_muji_idvan.jpg,True,True,20230816_AG_teaser_ag_sp_muji_idvan,20230816_AG_portrait_ag_sp_muji_idvan,Familie Patriotismus Gesundheit
107,02e,13,Santhiyapillai,Gabriel,Windisch,"Interkult. Übersetzer, Sakristan, Einwohnerrat",1957,0,,SP GfV – Gemeinsam für Vielfalt,...,Lohn glich für alle,uaS3t,"<iframe title=""Steckbrief Gabriel Santhiyapill...","<div style=""min-height:324px""><script type=""te...",ag_sp_santhiyapillai_gabriel.jpg,True,True,20230816_AG_teaser_ag_sp_santhiyapillai_gabriel,20230816_AG_portrait_ag_sp_s

In [8]:
### Schauen, ob es Worte gibt, die nicht vorkommen

df = pd.DataFrame({'Wort' : worte_umwelt, 'Anzahl':0})

#Jetzt werden die Antworten jedes Kandidierenden nach den Stichworten durchsucht und dementsprechend eine Gewichtung für jedes Thema zugewiesen
for index in tabelle_kandidierende.index:
    text = tabelle_kandidierende["svy_q1"][index]+tabelle_kandidierende["svy_q2"][index]+tabelle_kandidierende["svy_q3"][index]+tabelle_kandidierende["svy_q4"][index]
    for item in df.index:
        #print(df['Wort'][item])
        if re.findall(df['Wort'][item], text):
            df['Anzahl'][item] += 1

C:\Users\domin\AppData\Local\Temp/ipykernel_25220/2717823961.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Anzahl'][item] += 1


In [9]:
df

,Wort,Anzahl
0,Umweltschutz,15
1,Nachhaltigkeit,8
2,Klimawandel,44
3,Biodiversität,11
4,Ökologie,2
5,ökologisch,9
6,Naturschutz,4
7,Ressourcen,14
8,Kreislaufwirtschaft,4
9,Verkehrswende,1


In [613]:
from collections import Counter
for index in tabelle_kandidierende.index:
    text = text+tabelle_kandidierende["svy_q2"][index]+tabelle_kandidierende["svy_q3"][index]+tabelle_kandidierende["svy_q4"][index]
split_it = text.split()
Counter = Counter(split_it)
most_occur = Counter.most_common(500)

for wort in most_occur:
    print(wort)

('und', 9041)
('der', 5109)
('die', 4819)
('in', 2980)
('für', 2901)
('ich', 2699)
('zu', 2111)
('ist', 1745)
('mich', 1630)
('mit', 1623)
('den', 1548)
('eine', 1539)
('im', 1298)
('ein', 1148)
('bin', 1117)
('Ich', 1081)
('von', 1058)
('als', 992)
('Aargau', 979)
('auch', 970)
('Schweiz', 933)
('nicht', 928)
('wir', 918)
('auf', 863)
('das', 837)
('es', 811)
('dass', 720)
('Die', 704)
('sind', 672)
('Politik', 623)
('Kanton', 618)
('sich', 565)
('des', 540)
('mir', 523)
('müssen', 512)
('an', 490)
('muss', 479)
('-', 475)
('möchte', 469)
('dem', 463)
('um', 460)
('unsere', 454)
('Menschen', 448)
('einen', 436)
('wie', 424)
('meiner', 420)
('einer', 411)
('werden', 411)
('mehr', 405)
('bei', 380)
('habe', 377)
('Lösungen', 376)
('Gesellschaft', 373)
('werden.', 336)
('seit', 327)
('Jahren', 327)
('braucht', 322)
('immer', 320)
('meine', 320)
('durch', 310)
('nur', 295)
('setze', 293)
('uns', 285)
('Das', 281)
('gute', 281)
('unserer', 279)
('alle', 277)
('oder', 274)
('Es', 274)
('ein

In [609]:
text

'Mein Name ist Benjamin Giezendanner (41) und ich wohne mit meiner Ehefrau und meinen drei Töchtern (3, 5 und 8) in Rothrist. Nachdem ich eine kaufmännische Lehre auf einer Bank absolviert habe, bildete ich mich betriebswirtschaftlich weiter und führe heute ein Unternehmen im Transport.In der Politik braucht es Menschen, die sich in der Gemeinschaft und der Gesellschaft engagieren. Menschen, die ihren Teil der Verantwortung übernehmen und an unsere Gesellschaft ein klein wenig von dem zurückgeben, was sie von ihr erhalten haben. Gerne nehme ich diese Aufgabe wahr.Die Schweiz, als kleines Land, muss auch zukünftig ein Sonderfall bleiben. Ein Sonderfall, welcher mit Eigenschaften der direkten Demokratie, der bewährten immerwährenden Neutralität, sowie der Freiheit und Unabhängigkeit der Bürgerinnen und Bürger, den Unterschied ausmacht.Als Ständerat sehe ich mich nicht als «historisches Fossil» sondern ich will mit meinem künftigen Einsatz in der kleinen Kammer dem Aargau eine starke Stim

In [619]:
    print(thema)

Patriotismus
Zuwanderung
Zusammenarbeit
Umwelt
Energie
Wirtschaft
Sozialwerke
Familie
Gesundheit
Gleichstellung
Landwirtschaft
Verkehr


In [632]:
code = [dict_themen_wortlisten.keys()]
item = [0]
zipped = zip(code, item)

print(dict(zipped))

TypeError: unhashable type: 'dict_keys'

In [624]:
dict_themen_wortlisten{0}

SyntaxError: invalid syntax (Temp/ipykernel_13124/2934917718.py, line 1)

In [627]:
values = [dict_themen_wortlisten[i] for i in dict_themen_wortlisten]
print(values)

[['Heimat*', 'Nation ', 'eidgenössisch', 'Freiheit', 'Tradition', 'Souveränität', 'Zusammenhalt', 'Sicherheit', 'Demokratie', 'Patriotismus', 'freie.*Schweiz', 'sichere.*Schweiz', 'unabhängige.*Schweiz', 'Sonderfall', 'Neutralität', 'Schweizer'], ['Migration*', '[k|K]riminelle[|r] Ausländer', 'Ausschaffung', 'Masseneinwanderung', 'Bevölkerungsaustausch', 'Härtefallklausel', 'Gefährder', 'Islam*', 'Asyl*'], ['Europa', 'EU', 'Beziehung', 'Zusammenarbeit', 'Fachkräftemangel'], ['Umweltschutz', 'Nachhaltigkeit', 'Klimawandel', 'Biodiversität', 'Ökologie', 'ökologisch', 'Naturschutz', 'Ressourcen', 'Kreislaufwirtschaft', 'Verkehrswende', 'Fussabdruck', 'grün', 'erneuerbar', 'umweltfreundlich', 'Atomausstieg', 'Klimakrise', 'Treibhausgas*', 'CO2', 'ÖV'], ['Kernkraft*', 'Atom*', '[e|E]rneuerbare[|n] Energien', 'Energie*', 'Solarenergie', 'Energiewende', 'erneuerbar', 'Klimakrise', 'Energieversorgung', 'Gas*', 'Wasserkraft*', 'AKW', '.*Kraftwerk'], ['Wirtschaftswachstum', 'Arbeitsmarkt', 'Expo

In [634]:
dict_themen_wortlisten.keys()

dict_keys(['Patriotismus', 'Zuwanderung', 'Zusammenarbeit', 'Umwelt', 'Energie', 'Wirtschaft', 'Sozialwerke', 'Familie', 'Gesundheit', 'Gleichstellung', 'Landwirtschaft', 'Verkehr'])

In [39]:
for index in tabelle_kandidierende.index:
    print(tabelle_kandidierende["Name"][index])

Suter
Wermuth
Brizzi
Hunziker
Brügger
Basler
Jenni
Burger
Binggeli
Capanni
Dietrich
Gökçen
Lanz
Bär
Stocker Kalberer
Engeli
Sutter
Sommer
Haefeli
Petrušić
Lüthold
Dickson
Novak
Guerne-Kieferndorf
Dogan
Suter
Covo
Del Fabro
Mullis
Barth
Erne
Walser
Taubert
Scholer
Rohrer
Nöthiger
Müri
Mikic
Miceli
Kuhn
Hubmann
Hotz
Gestmann
Fedeli
Fondado
Erni
Cadinu
Allenspach
Arezki
Deck
Fankhauser
Feusi
Graf-Bruppacher
Hurtado
Keller
Klopfenstein
Kremer
Meier
Merz
Schöner
Stierli
Villiger
Widmer
Vinca
Riss
Muji
Schmid
Pascolin
Schmidmeister
Isik
Ammann
Kaufmann
Klopfenstein
Türkkan
Bertschi
Santhiyapillai
Silbereisen
Eicken
Langmoen
Ahmeti
Drozdov
Eichenberger Bryner
Fortunato
Glaab
Haller
Hasenfratz
Hofer
Leimgruber
Lindner
Marti
Mosimann
Schmid-Brem
Spiess
Syed
Zobrist
Anneler
Bächli
Christen
Grundisch
Janser
Jenni
Kocher
Krähenbühl
Lang
Morgenthaler
Neuhaus
Nježic
Perin
Scheidegger
Stojmenovski
Uhlig


In [54]:
df_verteilung = pd.DataFrame(columns=["Vorname","Name","Partei","Schwerpuntke","Patriotismus","Zuwanderung","Zusammenarbeit","Umwelt","Energie","Wirtschaft","Sozialwerke","Familie","Gesundheit","Gleichstellung","Landwirtschaft","Verkehr"])
for index in tabelle_kandidierende.index:
    df_verteilung["Name"] = tabelle_kandidierende["Name"][index]

IndexError: index 32 is out of bounds for axis 0 with size 0

In [53]:
df_verteilung

,Vorname,Name,Partei,Schwerpuntke,Patriotismus,Zuwanderung,Zusammenarbeit,Umwelt,Energie,Wirtschaft,Sozialwerke,Familie,Gesundheit,Gleichstellung,Landwirtschaft,Verkehr
Name,Uhlig,Uhlig,Uhlig,Uhlig,Uhlig,Uhlig,Uhlig,Uhlig,Uhlig,Uhlig,Uhlig,Uhlig,Uhlig,Uhlig,Uhlig,Uhlig


In [ ]:
#In diesem Dict werden die Wortlisten den Themen zugewiesen
dict_themen_wortlisten = {
    "Patriotismus":worte_patriotismus,
    "Zuwanderung":worte_zuwanderung,
    "Zusammenarbeit":worte_zusammenarbeit,
    "Umwelt":worte_umwelt,
    "Energie":worte_energie,
    "Wirtschaft":worte_wirtschaft,
    "Sozialwerke":worte_sozialwerke,
    "Familie":worte_familie,
    "Gesundheit":worte_gesundheit,
    "Gleichstellung":worte_gleichstellung,
    "Landwirtschaft":worte_landwirtschaft,
    "Verkehr":worte_verkehr
}

#Jetzt werden die Antworten jedes Kandidierenden nach den Stichworten durchsucht und dementsprechend eine Gewichtung für jedes Thema zugewiesen
for index in tabelle_kandidierende.index:
    #Alle Zähler auf 0 setzen
    dict_themen = {
    "Patriotismus":0,
    "Zuwanderung":0,
    "Zusammenarbeit":0,
    "Umwelt":0,
    "Energie":0,
    "Wirtschaft":0,
    "Sozialwerke":0,
    "Familie":0,
    "Gesundheit":0,
    "Gleichstellung":0,
    "Landwirtschaft":0,
    "Verkehr":0
    }
    text = tabelle_kandidierende["svy_q2"][index]+tabelle_kandidierende["svy_q3"][index]+tabelle_kandidierende["svy_q4"][index]
    for item in dict_themen.copy():
    #for item in dict_themen:
        for wort in dict_themen_wortlisten[item]:
            try:
                #if re.findall(wort, text, re.IGNORECASE):
                if re.findall(wort, text):
                    #print(item)
                    dict_themen[item] += 1
            except:
                print("FEHLER")
                print(item)
        if dict_themen[item] == 0:
            del dict_themen[item]
            #print("0")
        else:
            #print(dict_themen[item],len(dict_themen_wortlisten[item]))
            dict_themen[item] = dict_themen[item]/len(dict_themen_wortlisten[item])


    #print(dict_themen)
    df_verteilung["Name"] = tabelle_kandidierende["Name"][index]
    print(tabelle_kandidierende["Name"][index])
    df_verteilung = df_verteilung.append(dict_themen,ignore_index=True)
    df = pd.DataFrame(list(dict_themen.items()), columns = ['Thema','Stärke'])
    schwerpunkte = df.sort_values('Stärke', ascending=False).head(3)
    text_schwerpunkte = ""
    for schwerpunkt in schwerpunkte.index:
        text_schwerpunkte = text_schwerpunkte+" "+schwerpunkte["Thema"][schwerpunkt]
    #tabelle_kandidierende.copy()["Schwerpunkte"][index] = text_schwerpunkte
    tabelle_kandidierende["Schwerpunkte"][index] = text_schwerpunkte
    #print(text_schwerpunkte)
#df_verteilung.loc[:, 'Name'] = tabelle_kandidierende.loc[:, 'Name']
df_verteilung.to_csv('Themen.csv', index=False)

In [102]:
dict_themen["Landwirtschaft"]

['Bauer', 'Landwirtschaft', 'Nahrung', 'Versorgungssicherheit']

In [108]:
for thema in dict_themen:
    print(thema)
    for wort in dict_themen[thema]:
        print(wort)
    print(len(dict_themen[thema]))

Patriotismus
Heimat*
Nation 
eidgenössisch
Freiheit
Tradition
Souveränität
Zusammenhalt
Sicherheit
Demokratie
Patriotismus
freie.*Schweiz
sichere.*Schweiz
unabhängige.*Schweiz
Sonderfall
Neutralität
Schweizer
16
Zuwanderung
Migration*
[k|K]riminelle[|r] Ausländer
Ausschaffung
Masseneinwanderung
Bevölkerungsaustausch
Härtefallklausel
Gefährder
Islam*
Asyl*
Flüchtling
10 Millionen Schweiz
11
Zusammenarbeit
Europa
EU
Beziehung
Zusammenarbeit
Fachkräftemangel
5
Umwelt
Umweltschutz
Nachhaltigkeit
Klimawandel
Biodiversität
Ökologie
ökologisch
Naturschutz
Ressourcen
Kreislaufwirtschaft
Verkehrswende
Fussabdruck
grün
erneuerbar
umweltfreundlich
Klimakrise
Treibhausgas*
CO2
ÖV
Antomausstieg
19
Energie
Kernkraft*
Atom*
[e|E]rneuerbare[|n] Energien
Energie*
Solarenergie
Energiewende
erneuerbar
Klimakrise
Energieversorgung
Gas*
Wasserkraft*
AKW
.*Kraftwerk
Antomausstieg
14
Wirtschaft
Wirtschaftswachstum
Arbeitsmarkt
Export
Binnenmarkt
Arbeitsplätze
Investition
Innovation
Wettbewerb
Handel
Steuer
R

In [205]:
for partei in dict_parteien:
    #print(dict_parteien[partei])
    tabelle_kandidierende = tabelle_kandidierende.loc[tabelle_kandidierende['ListeNr'].str.contains(dict_parteien[partei], case=False)]
    make_df(partei)

In [248]:
partei="SVP"
tabelle_kandidierende = tabelle_kandidierende.loc[tabelle_kandidierende['ListeNr'].str.contains(dict_parteien[partei], case=False)]
tabelle_kandidierende

,Name,Vorname,Partei,ListeNr,svy_q2,svy_q3,svy_q4,Patriotismus,Zuwanderung,Zusammenarbeit,Umwelt,Energie,Wirtschaft,Sozialwerke,Familie,Gesundheit,Gleichstellung,Landwirtschaft,Verkehr,Schwerpunkte
1,Abt,Lukas,Junge_SVP,01b,,,,0.0000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.00,0.000000,
33,Barp,Miro,SVP,01a,"Im Beruf sehe ich, dass das Gesundheitswesen i...",Ohne Kurskorrektur steuern wir einer 10-Mio.-S...,"Ich will, dass der Aargau als viertgrösster Ka...",0.0625,0.000000,0.0,0.000000,0.071429,0.00,0.000000,0.076923,0.043478,0.0,0.00,0.000000,
62,Bircher,Martina,SVP,01a,Ich setzte mich für unsere Schweiz ein damit u...,Es braucht eine neue Migrations- und Asylpolit...,"Selbstbewusster Kanton, der seine Vorzüge einb...",0.0000,0.363636,0.0,0.000000,0.000000,0.00,0.000000,0.076923,0.086957,0.0,0.00,0.000000,
64,Blülle,Benjamin,Junge_SVP,01b,Die Jungen werden in Zukunft die Politik mache...,Überlastung in unserem Land durch den enormen ...,Die Gemeinden von den Sozialhilfeausgaben entl...,0.1250,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.00,0.000000,
71,Borer-Mathys,Barbara,SVP,01a,Mein Motto ist: «Nicht träumen – machen!». Ich...,An erster Stelle steht die Begrenzung der Zuwa...,"Parlament und Verwaltung müssen effizienter, s...",0.1250,0.000000,0.0,0.000000,0.000000,0.00,0.166667,0.000000,0.043478,0.0,0.00,0.000000,
89,Büchler,Leonardo,Junge_SVP,01b,,,,0.0000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.00,0.000000,
97,Burger,Nicole,SVP,01a,Die letzten Jahre haben mich aufgerüttelt. Pla...,Die Schweiz steht mit ihren bewährten Werten w...,Unser politisches System basiert u.a. auf Föde...,0.2500,0.000000,0.2,0.000000,0.000000,0.05,0.000000,0.000000,0.000000,0.0,0.00,0.000000,
99,Burgherr,Thomas,SVP,01a,Ich kenne als Unternehmer nicht nur den Alltag...,"Die grösste Herausforderung besteht darin, das...",Möglichst viel Föderalismus und weniger Zentra...,0.0625,0.000000,0.0,0.000000,0.000000,0.05,0.000000,0.000000,0.000000,0.0,0.00,0.000000,
189,Furrer,Samira,Junge_SVP,01b,,,,0.0000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.00,0.000000,
191,Gadient,Daniel,Junge_SVP,01b,,,,0.0000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.00,0.000000,


In [210]:
tabelle_kandidierende = tabelle_kandidierende.loc[tabelle_kandidierende['ListeNr'].str.contains("01", case=False)]
tabelle_kandidierende

,Name,Vorname,Partei,ListeNr,svy_q2,svy_q3,svy_q4,Patriotismus,Zuwanderung,Zusammenarbeit,Umwelt,Energie,Wirtschaft,Sozialwerke,Familie,Gesundheit,Gleichstellung,Landwirtschaft,Verkehr


In [225]:
tabelle_kandidierende["Patriotismus"][666]

0.0625

In [229]:
mini_dict

{'Verkehr': 0.16666666666666666}

In [233]:
tabelle_kandidierende = pd.read_csv('kandidierende.csv') 
tabelle_kandidierende

,ListeNr,WahlzettelNr,Name,Vorname,Wohnort,Beruf,Jahrgang,bisher,Amtsjahr,Parteibezeichnung,...,svy_q3,svy_q4,dw_id,iframe_full,iframe_webcomponent,photo_file_path,has_teaser,has_portrait,photo_id_teaser,photo_id_portrait
0,07g,11,Abrham,Sarah,Aesch,Oberstufenlehrerin,1992,0,NaN,engagiert. für eine gute Bildung (enB),...,NaN,NaN,Xo6aE,"<iframe title=""Steckbrief Sarah Abrham"" aria-l...","<div style=""min-height:324px""><script type=""te...",ag_evp_abrham_sarah.jpg,False,False,NaN,NaN
1,01b,9,Abt,Lukas,Brittnau,"Landwirt, Student Agronomie, Kompaniekommandant",1997,0,NaN,JSVP – Junge Schweizerische Volkspartei,...,NaN,NaN,EojZC,"<iframe title=""Steckbrief Lukas Abt"" aria-labe...","<div style=""min-height:324px""><script type=""te...",ag_junge_svp_abt_lukas.jpg,True,True,20230816_AG_teaser_ag_junge_svp_abt_lukas,20230816_AG_portrait_ag_junge_svp_abt_lukas
2,03e,1,Ackermann,Adrian,Kaisten,"Unternehmer, ehem. Grossrat, Bezirksrichter",1964,0,NaN,FDP Unt. – Liberal für Unternehmen,...,NaN,NaN,UwXGL,"<iframe title=""Steckbrief Adrian Ackermann"" ar...","<div style=""min-height:324px""><script type=""te...",ag_fdp_ackermann_adrian.jpg,True,True,20230816_AG_teaser_ag_fdp_ackermann_adrian,20230816_AG_portrait_ag_fdp_ackermann_adrian
3,07b,5,Acklin-Stihl,Jasmin,Frick,"Dipl. Pflegefachfrau HF, Schwimmlehrerin, Mutter",1988,0,NaN,engagiert. JEVP We. – Junge Evangelische Volks...,...,Neben der Bildungspolitik ist ein grosser Baus...,- Verbesserung der Gesundheitspolitik - Offenh...,knnh5,"<iframe title=""Steckbrief Jasmin Acklin-Stihl""...","<div style=""min-height:324px""><script type=""te...",ag_junge_evp_acklin-stihl_jasmin.jpg,True,True,20230816_AG_teaser_ag_junge_evp_acklin-stihl_j...,20230816_AG_portrait_ag_junge_evp_acklin-stihl...
4,06g,6,Aegerter,Emanuel,Wettingen,Student der Rechtswissenschaften,1997,0,NaN,GLP-E – Gesellschaft und Engagement,...,NaN,NaN,evrcj,"<iframe title=""Steckbrief Emanuel Aegerter"" ar...","<div style=""min-height:324px""><script type=""te...",ag_glp_aegerter_emanuel.jpg,True,True,20230816_AG_teaser_ag_glp_aegerter_emanuel,20230816_AG_portrait_ag_glp_aegerter_emanuel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,03b,15,Zingg,Lionel,Wohlen,"Student Politik und Wirtschaft, Einwohnerrat",2000,0,NaN,FDP.Die Liberalen – Jungfreisinnige,...,NaN,NaN,Qy31H,"<iframe title=""Steckbrief Lionel Zingg"" aria-l...","<div style=""min-height:324px""><script type=""te...",ag_fdp_zingg_lionel.jpg,True,True,20230816_AG_teaser_ag_fdp_zingg_lionel,20230816_AG_portrait_ag_fdp_zingg_lionel
709,02f,16,Zobrist,Nico Julian,Unterentfelden,Student Humanmedizin,2001,0,NaN,SP GE – Gesundheit,...,"Die Klimakrise ist unser grösstes Problem, das...",Als Nationalrat steht das Wohle der gesamten S...,MUE4Z,"<iframe title=""Steckbrief Nico Julian Zobrist""...","<div style=""min-height:324px""><script type=""te...",ag_sp_zobrist_nico_julian.jpg,True,True,20230816_AG_teaser_ag_sp_zobrist_nico_julian,20230816_AG_portrait_ag_sp_zobrist_nico_julian
710,05c,16,Züger Cáceres,Regula,Lenzburg,"Dipl. Ing. Agr. ETH, Einwohnerrätin",1968,0,NaN,GRÜNE Bio-Bäuerinnen*,...,NaN,NaN,hWn2H,"<iframe title=""Steckbrief Regula Züger Cáceres...","<div style=""min-height:324px""><script type=""te...",ag_gruene_zueger_caceres_regula.jpg,True,True,20230816_AG_teaser_ag_gruene_zueger_caceres_re...,20230816_AG_portrait_ag_gruene_zueger_caceres_...
711,06c,4,Zumstein,Antonia,Wettingen,"Kauffrau, Einwohnerrätin",1962,0,NaN,GLP-S – Grünliberale Seniorinnen und Senioren,...,Der Klimawandel geht an der Schweiz nicht spur...,Die Ausrichtung des Aargaus in der Energieprod...,UHm06,"<iframe title=""Steckbrief Antonia Zumstein"" ar...","<div style=""min-height:324px""><script type=""te...",ag_glp_zumstein_antonia.jpg,True,True,20230816_AG_teaser_ag_glp_zumstein_antonia,20230816_AG_portrait_ag_glp_zumstein_antonia


In [247]:
tabelle_kandidierende

,Name,Vorname,Partei,ListeNr,svy_q2,svy_q3,svy_q4,Patriotismus,Zuwanderung,Zusammenarbeit,Umwelt,Energie,Wirtschaft,Sozialwerke,Familie,Gesundheit,Gleichstellung,Landwirtschaft,Verkehr,Schwerpunkte
0,Abrham,Sarah,EVP,07g,,,,0.0,0.0,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.0,0.00,0.0,
1,Abt,Lukas,Junge_SVP,01b,,,,0.0,0.0,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.0,0.00,0.0,
2,Ackermann,Adrian,FDP,03e,,,,0.0,0.0,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.0,0.00,0.0,
3,Acklin-Stihl,Jasmin,Junge_EVP,07b,In der Politik gibt es für mich drei Herzensan...,Neben der Bildungspolitik ist ein grosser Baus...,- Verbesserung der Gesundheitspolitik - Offenh...,0.0,0.0,0.0,0.000000,0.000000,0.00,0.0,0.230769,0.086957,0.0,0.25,0.0,
4,Aegerter,Emanuel,GLP,06g,,,,0.0,0.0,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.0,0.00,0.0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,Zingg,Lionel,FDP,03b,,,,0.0,0.0,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.0,0.00,0.0,
709,Zobrist,Nico Julian,SP,02f,Ich will mich auch in Bern einsetzen für eine ...,"Die Klimakrise ist unser grösstes Problem, das...",Als Nationalrat steht das Wohle der gesamten S...,0.0,0.0,0.2,0.095238,0.071429,0.00,0.0,0.000000,0.043478,0.0,0.00,0.0,
710,Züger Cáceres,Regula,Gruene,05c,,,,0.0,0.0,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.0,0.00,0.0,
711,Zumstein,Antonia,GLP,06c,"Der Erhalt unserer wertvollen Natur, das fried...",Der Klimawandel geht an der Schweiz nicht spur...,Die Ausrichtung des Aargaus in der Energieprod...,0.0,0.0,0.0,0.047619,0.142857,0.05,0.0,0.000000,0.000000,0.0,0.00,0.0,


Patriotismus
Zuwanderung
Umwelt
Energie
Wirtschaft
Sozialwerke
Familie
Gesundheit
Gleichstellung
Landwirtschaft
Verkehr
Liberalismus
Familie


In [313]:
dict_schwerpunkte

{'Patriotismus': 1.0,
 'Zuwanderung': 2.0,
 'Umwelt': 0.0,
 'Energie': 1.0,
 'Wirtschaft': 0.0,
 'Sozialwerke': 0.0,
 'Familie': 4.0,
 'Gesundheit': 0.0,
 'Gleichstellung': 0.0,
 'Landwirtschaft': 0.0,
 'Verkehr': 1.0,
 'Liberalismus': nan}

In [337]:
dict_schwerpunkte = {}
for thema in dict_themen:
    dict_schwerpunkte[thema] = tabelle_kandidierende[thema][0]
    highest = max(dict_schwerpunkte.values())
    
    try:
        schwerpunkt = ([key for key in dict_schwerpunkte if dict_schwerpunkte[key] == highest][2])+", "+([key for key in dict_schwerpunkte if dict_schwerpunkte[key] == highest][1])+", "+([key for key in dict_schwerpunkte if dict_schwerpunkte[key] == highest][0])
    except:
        try:
            schwerpunkt = ([key for key in dict_schwerpunkte if dict_schwerpunkte[key] == highest][1])+", "+([key for key in dict_schwerpunkte if dict_schwerpunkte[key] == highest][0])
        except:
            try:
                schwerpunkt = ([key for key in dict_schwerpunkte if dict_schwerpunkte[key] == highest][0])
            except:
                schwerpunkt = ""

['Familie']


In [339]:
dict_schwerpunkte

{'Patriotismus': 0.0,
 'Zuwanderung': 0.0,
 'Umwelt': 0.0,
 'Energie': 0.0,
 'Wirtschaft': 0.0,
 'Sozialwerke': 0.0,
 'Familie': 0.0,
 'Gesundheit': 0.0,
 'Gleichstellung': 0.0,
 'Landwirtschaft': 0.0,
 'Verkehr': 0.0,
 'Liberalismus': 0.0}

In [341]:
highest = max(dict_schwerpunkte.values())
highest

0.0

In [307]:
db_schwerpunkte

{'punkte', 'thema'}

In [7]:
tabelle_kandidierende

,Name,Vorname,Partei,ListeNr,svy_q2,svy_q3,svy_q4,Patriotismus,Zuwanderung,Umwelt,Energie,Wirtschaft,Sozialwerke,Familie,Gesundheit,Gleichstellung,Landwirtschaft,Verkehr,Liberalismus,Schwerpunkt
0,Abrham,Sarah,EVP,07g,,,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,Abt,Lukas,Junge_SVP,01b,,,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,Ackermann,Adrian,FDP,03e,,,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,Acklin-Stihl,Jasmin,Junge_EVP,07b,In der Politik gibt es für mich drei Herzensan...,Neben der Bildungspolitik ist ein grosser Baus...,- Verbesserung der Gesundheitspolitik - Offenh...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0,0.0,0.0,Familie
4,Aegerter,Emanuel,GLP,06g,,,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,Zingg,Lionel,FDP,03b,,,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
709,Zobrist,Nico Julian,SP,02f,Ich will mich auch in Bern einsetzen für eine ...,"Die Klimakrise ist unser grösstes Problem, das...",Als Nationalrat steht das Wohle der gesamten S...,0.0,1.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,Umwelt
710,Züger Cáceres,Regula,Gruene,05c,,,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
711,Zumstein,Antonia,GLP,06c,"Der Erhalt unserer wertvollen Natur, das fried...",Der Klimawandel geht an der Schweiz nicht spur...,Die Ausrichtung des Aargaus in der Energieprod...,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Energie


,Thema,Anzahl
0,Patriotismus,6.0
1,Zuwanderung,12.0
2,Umwelt,6.0
3,Energie,6.0
4,Wirtschaft,8.0
5,Sozialwerke,7.0
6,Familie,5.0
7,Gesundheit,9.0
8,Gleichstellung,0.0
9,Landwirtschaft,0.0


In [47]:
tabelle_kandidierende


,Name,Vorname,Partei,ListeNr,svy_q2,svy_q3,svy_q4,Patriotismus,Zuwanderung,Umwelt,Energie,Wirtschaft,Sozialwerke,Familie,Gesundheit,Gleichstellung,Landwirtschaft,Verkehr,Liberalismus,Schwerpunkt


In [ ]:
 
# Use the loc method to add the new row to the DataFrame
